In [1]:
import fitz
import numpy as np
import pandas as pd
from tqdm import tqdm

import io
from PIL import Image, ImageDraw, ImageFont, ImageColor

import math
import re

In [ ]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

: 

In [3]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

vol1_index = list(range(616, 639)) #inclusive
vol2_index = list(range(703, 725))
vol3_index = list(range(555, 583))

In [4]:
def copy_data(df, src_index, dst_indecies, col_name, inplace = True):
    """ used for modifying columns of datatype that can be dirrectly assigned to multiple rows
        INPUT: 
              - df: target dataframe to modify 
              - 
        OUTPUT: returns df 
    """
    if inplace == False:
        df = df.copy()
    
    src_value = df.loc[src_index, col_name]
    if isinstance(src_value, tuple) or isinstance(src_value, list):
        df.loc[dst_indecies, col_name] = [src_value] * len(dst_indecies)
    else: 
        df.loc[dst_indecies, col_name] = src_value

    return df 

In [5]:
def get_bbox(df, src_indecies, indecies, col_name, inplace = True):
    x0 = df[col_name].apply(lambda x : x[0]).min()
    y0 = df[col_name].apply(lambda x : x[1]).min()
    x1 = df[col_name].apply(lambda x : x[2]).max()
    y1 = df[col_name].apply(lambda x : x[3]).max()
    bbox = (x0, y0, x1, y1)
    df.loc[indecies, col_name] = [bbox] * len(indecies)

    return df

In [6]:
def reiterate(df, src_index, dst_indecies, col_name, inplace = True):
    """ Used for modifying columns with lists/tuples
        INPUT: 
              - df: target dataframe to modify 
              - 
        OUTPUT: returns df 
    """
    if inplace == False:
        df = df.copy()
    
    df.loc[dst_indecies, col_name] = list(range(len(dst_indecies)))

    return df 

In [7]:
defaul_col_transform = {
                        "vol_num" : lambda x : x
                       }

In [ ]:
def transform_data(df,src_index, indecies, col_transform_dict, inplace = True):
    """INPUT
        - col_transform_dict: dictionary with column names as keys and transformation function as values"""